#### The requests package is installed

In [1]:
!pip install requests

Defaulting to user installation because normal site-packages is not writeable


#### The BeautifulSoup and Requests library are imported

In [267]:
from bs4 import BeautifulSoup
import requests
import time
from time import sleep
import pandas as pd
import numpy as np
pd.options.display.max_columns=None

collection=dread_guitars.find_all('div', class_="fx-product-list-entry")
type(collection)

In [264]:
url_list=[]

def get_url():
    
    global base_url
    
    base_url = 'https://www.thomann.de/gb/dreadnought_guitars.html?ls=50&pg=1&gk=GIWEDR&cme=false'
    
    for i in range(2,3):
            url_split=base_url.split('pg=')
            url_part2 =url_split[1]
            url_next_page=url_split[0] + 'pg=' + url_part2.replace('1',str(i))
            
            url_list.append(url_next_page)
            
    url_list.insert(0,base_url) 
    
    for i in url_list:
        print(i)

In [265]:
def get_guitar_details():
    
    global Guitar_Feature_Type_list
    global Guitar_Feature_Name_list
    global soup
    global soup2
    global guitar_details
    
    
    get_url()
    
    guitar_details=[]
    
    
    
    for i in url_list:
        
        result=requests.get(i, timeout=5, headers={'Connection':'close'})

        print('Response Status Code:  ' + str(result.status_code))

        src=result.text
        
        result.close()

        soup= BeautifulSoup(src,'lxml') 

        dread_guitars=soup.find('div',class_ = "js-search")

        collection=dread_guitars.find_all('div', class_="fx-product-list-entry")

        


        for guitar in collection:

                Product=guitar.find('a', class_='product__content')

                Manufacturer=Product.find('span', class_='title__manufacturer').text
                
                Model=Product.find('span', class_='title__name').text
                
                Price_unclean=Product.find('div', class_= 'product__price').text

                if Price_unclean is not None:
                    
                    Price_unclean.strip()

                    Price=Price_unclean.strip().lstrip('£')
                
                else:
                    
                    Price= None
                

                try:
                    Product_availability_unclean=Product.find('div',class_='product__availability fx-availability fx-availability--in-stock').text

                    Product_availability=Product_availability_unclean.strip()

                except:
                    Product_availability= None
                
                
                
                link_part2=guitar.find('a', class_='product__content')['href']
                
                if link_part2 is not None:
                    
                        link_product='https://www.thomann.de/gb/' + link_part2 


                        result2=requests.get(link_product,timeout=5, headers={'Connection':'close'})

                        src2=result2.text

                        result2.close()


                        soup2=BeautifulSoup(src2,'lxml')

                        for i in soup2.find_all('div',class_='keyfeature keyfeature--hidden fx-collapsible__content'):
                            i.decompose()

                        for i in soup2.find_all('div',class_='keyfeature keyfeature--toggle-trigger'):
                            i.decompose()



                        try:

                            guitar_info=soup2.find('div', class_='fx-grid__col fx-col--12 fx-col--lg-4')
                            guitar_rank=guitar_info.find('span', class_='meta-box__value').text.strip()


                        except:

                            guitar_rank=None


                        parent=soup2.find('div', class_='badges')

                        try:

                            for i,j in enumerate(parent):
                                if i==3:
                                    years_warranty=j.text

                            index=years_warranty.find('-')
                            years_warranty=years_warranty[index-1]

                        except:

                            years_warranty=None



                        key_features=soup2.find('div', class_='keyfeatures')

                        try:

                                feature_type=key_features.find_all('span', class_='keyfeature__label')
                                feature_name=key_features.find_all('span', class_ ='fx-text fx-text--bold fx-text--no-margin')

                                Guitar_Feature_Type_list=[]
                                Guitar_Feature_Name_list=[]

                                for i in feature_type:
                                    x=i.text
                                    Guitar_Feature_Type_list.append(x)

                                for j in feature_name:
                                    y=j.text
                                    Guitar_Feature_Name_list.append(y)

                        except:

                                Guitar_Feature_Type_list.append(None)
                                Guitar_Feature_Name_list.append(None)


                        entry={}

                        for i,j in zip(Guitar_Feature_Type_list,Guitar_Feature_Name_list):

                                entry['Manufacturer']= Manufacturer
                                entry['Model']= Model
                                entry['Price']= Price
                                entry['Product_availability']= Product_availability
                                entry['guitar_rank']= guitar_rank
                                entry['Years_Warranty']=years_warranty

                                entry[i.upper()]=j

                        guitar_details.append(entry)

                        time.sleep(3)
                
                else:
                        print('Product Link not available')
                        
                        entry={}
                        entry['Manufacturer']= Manufacturer
                        entry['Model']= Model
                        entry['Price']= Price
                        entry['Product_availability']= Product_availability
                        
                        guitar_details.append(entry)
                        
                    

In [266]:
get_guitar_details()

https://www.thomann.de/gb/dreadnought_guitars.html?ls=50&pg=1&gk=GIWEDR&cme=false
https://www.thomann.de/gb/dreadnought_guitars.html?ls=50&pg=2&gk=GIWEDR&cme=false
Response Status Code:  200
Response Status Code:  200


In [268]:
df3=pd.json_normalize(guitar_details)

In [269]:
df3.shape

(100, 30)

In [274]:
df=df3.copy()

In [275]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Manufacturer            100 non-null    object
 1   Model                   100 non-null    object
 2   Price                   100 non-null    object
 3   Product_availability    81 non-null     object
 4   guitar_rank             100 non-null    object
 5   Years_Warranty          100 non-null    object
 6   AVAILABLE SINCE         100 non-null    object
 7   ITEM NUMBER             100 non-null    object
 8   SALES UNIT              100 non-null    object
 9   TOP                     99 non-null     object
 10  BACK AND SIDES          99 non-null     object
 11  CUTAWAY                 99 non-null     object
 12  FRETBOARD               99 non-null     object
 13  NUT WIDTH IN MM         99 non-null     object
 14  FRETS                   99 non-null     object
 15  PICKUPS

In [276]:
df.head()

,Manufacturer,Model,Price,Product_availability,guitar_rank,Years_Warranty,AVAILABLE SINCE,ITEM NUMBER,SALES UNIT,TOP,BACK AND SIDES,CUTAWAY,FRETBOARD,NUT WIDTH IN MM,FRETS,PICKUPS,COLOUR,CASE,INCL. GIGBAG,DESIGN,POWER IN WATTS,REVERB,EFFECTS,HEADPHONE CONNECTION,INCL. CASE / GIGBAG,INCL. CABEL,INCL. PICKS,INCL. TUNER,INCL. STRAP,INCL. EDUCATIONAL BOOK
0,Fender,CD-60S Black WN,145,In stock,60,3,January 2019,455910,1 piece(s),"Spruce, Solid",Mahogany,No,Walnut,"43,00 mm",20,No,Black,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Harley Benton,D-120CE BK,75,In stock,177,3,January 2003,157819,1 piece(s),Spruce,"Sapelli, Mahogany",Yes,Roseacer,"42,50 mm",20,Yes,Black,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Harley Benton,CLD-60SCE TB,111,In stock,61,3,December 2020,500960,1 piece(s),"Spruce, Solid",Mahogany,Yes,Roseacer,"43,00 mm",20,Yes,Blue,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fender,CD-60 NA V3,111,In stock,29,3,September 2006,190655,1 piece(s),Spruce,Mahogany,No,Walnut,"43,00 mm",20,No,Natural,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Harley Benton,D-120CE NT Bundle,85,In stock,16,3,June 2017,416052,1 piece(s),Spruce,"Sapelli, Mahogany",Yes,Roseacer,"42,50 mm",20,Yes,Natural,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [277]:
df.isnull().sum()

Manufacturer               0
Model                      0
Price                      0
Product_availability      19
guitar_rank                0
Years_Warranty             0
AVAILABLE SINCE            0
ITEM NUMBER                0
SALES UNIT                 0
TOP                        1
BACK AND SIDES             1
CUTAWAY                    1
FRETBOARD                  1
NUT WIDTH IN MM            1
FRETS                      1
PICKUPS                    2
COLOUR                     0
CASE                       1
INCL. GIGBAG               1
DESIGN                    99
POWER IN WATTS            99
REVERB                    99
EFFECTS                   99
HEADPHONE CONNECTION      99
INCL. CASE / GIGBAG       99
INCL. CABEL               99
INCL. PICKS               99
INCL. TUNER               99
INCL. STRAP               99
INCL. EDUCATIONAL BOOK    99
dtype: int64

In [278]:
df[df['PICKUPS'].isnull()==True]

,Manufacturer,Model,Price,Product_availability,guitar_rank,Years_Warranty,AVAILABLE SINCE,ITEM NUMBER,SALES UNIT,TOP,BACK AND SIDES,CUTAWAY,FRETBOARD,NUT WIDTH IN MM,FRETS,PICKUPS,COLOUR,CASE,INCL. GIGBAG,DESIGN,POWER IN WATTS,REVERB,EFFECTS,HEADPHONE CONNECTION,INCL. CASE / GIGBAG,INCL. CABEL,INCL. PICKS,INCL. TUNER,INCL. STRAP,INCL. EDUCATIONAL BOOK
76,Epiphone,Slash J-45 November Burst,694,None,58,3,December 2021,518609,1 piece(s),"Spruce, Solid",Mahogany,No,indian Laurel,"43,00 mm",20,NaN,Sunburst,Yes,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,Harley Benton,HBD120CEBK Bundle,103,In stock,43,3,March 2007,111395,1 piece(s),NaN,NaN,NaN,NaN,NaN,NaN,NaN,Black,NaN,NaN,Dreadnought,0 W,No,No,No,Yes,No,Yes,Yes,No,Yes


In [279]:
df['PICKUPS'].fillna('No',inplace=True)

In [280]:
df.isnull().sum()

Manufacturer               0
Model                      0
Price                      0
Product_availability      19
guitar_rank                0
Years_Warranty             0
AVAILABLE SINCE            0
ITEM NUMBER                0
SALES UNIT                 0
TOP                        1
BACK AND SIDES             1
CUTAWAY                    1
FRETBOARD                  1
NUT WIDTH IN MM            1
FRETS                      1
PICKUPS                    0
COLOUR                     0
CASE                       1
INCL. GIGBAG               1
DESIGN                    99
POWER IN WATTS            99
REVERB                    99
EFFECTS                   99
HEADPHONE CONNECTION      99
INCL. CASE / GIGBAG       99
INCL. CABEL               99
INCL. PICKS               99
INCL. TUNER               99
INCL. STRAP               99
INCL. EDUCATIONAL BOOK    99
dtype: int64

In [281]:
df[df['FRETBOARD'].isnull()==True]

,Manufacturer,Model,Price,Product_availability,guitar_rank,Years_Warranty,AVAILABLE SINCE,ITEM NUMBER,SALES UNIT,TOP,BACK AND SIDES,CUTAWAY,FRETBOARD,NUT WIDTH IN MM,FRETS,PICKUPS,COLOUR,CASE,INCL. GIGBAG,DESIGN,POWER IN WATTS,REVERB,EFFECTS,HEADPHONE CONNECTION,INCL. CASE / GIGBAG,INCL. CABEL,INCL. PICKS,INCL. TUNER,INCL. STRAP,INCL. EDUCATIONAL BOOK
92,Harley Benton,HBD120CEBK Bundle,103,In stock,43,3,March 2007,111395,1 piece(s),NaN,NaN,NaN,NaN,NaN,NaN,No,Black,NaN,NaN,Dreadnought,0 W,No,No,No,Yes,No,Yes,Yes,No,Yes


In [282]:
df.dropna(axis=0, subset= 'FRETBOARD', inplace=True)

In [283]:
df.isnull().sum()

Manufacturer               0
Model                      0
Price                      0
Product_availability      19
guitar_rank                0
Years_Warranty             0
AVAILABLE SINCE            0
ITEM NUMBER                0
SALES UNIT                 0
TOP                        0
BACK AND SIDES             0
CUTAWAY                    0
FRETBOARD                  0
NUT WIDTH IN MM            0
FRETS                      0
PICKUPS                    0
COLOUR                     0
CASE                       0
INCL. GIGBAG               0
DESIGN                    99
POWER IN WATTS            99
REVERB                    99
EFFECTS                   99
HEADPHONE CONNECTION      99
INCL. CASE / GIGBAG       99
INCL. CABEL               99
INCL. PICKS               99
INCL. TUNER               99
INCL. STRAP               99
INCL. EDUCATIONAL BOOK    99
dtype: int64

In [284]:
column_list=['SALES UNIT','DESIGN','POWER IN WATTS','REVERB','EFFECTS','HEADPHONE CONNECTION','INCL. CASE / GIGBAG','INCL. CABEL','INCL. PICKS','INCL. TUNER','INCL. STRAP','INCL. EDUCATIONAL BOOK']

for i in column_list:
    if i in list(df.columns):
        df.drop(i, axis=1, inplace=True)

In [285]:
for column in list(df.columns):
    df.rename(columns = {column : column.upper()} , inplace = True)

In [286]:
df['PRODUCT_AVAILABILITY'].fillna(value='Unavailable', inplace=True)

In [287]:
df.isnull().sum()

MANUFACTURER            0
MODEL                   0
PRICE                   0
PRODUCT_AVAILABILITY    0
GUITAR_RANK             0
YEARS_WARRANTY          0
AVAILABLE SINCE         0
ITEM NUMBER             0
TOP                     0
BACK AND SIDES          0
CUTAWAY                 0
FRETBOARD               0
NUT WIDTH IN MM         0
FRETS                   0
PICKUPS                 0
COLOUR                  0
CASE                    0
INCL. GIGBAG            0
dtype: int64

In [288]:
df['YEARS_WARRANTY'].unique()

array(['3'], dtype=object)

In [289]:
df['MANUFACTURER'].unique()

array(['Fender', 'Harley Benton', 'Epiphone', 'Takamine', 'DAngelico',
       'Gibson', 'Squier', 'Taylor', 'Guild', 'Ibanez', 'Seagull', 'Cort',
       'Yamaha', 'Recording King'], dtype=object)

In [290]:
df['TOP'].unique()

array(['Spruce, Solid', 'Spruce', 'Mahogany', 'Cedar', 'Mahogany, Solid',
       'Cedar, Solid', 'Ebony', 'Coa solid', 'Sitka Spruce solid'],
      dtype=object)

In [291]:
df['BACK AND SIDES'].unique()

array(['Mahogany', 'Sapelli, Mahogany', 'Maple', 'Walnut', 'Koa, Solid',
       'Solid Mahogany', 'Walnut, Solid', 'Nato', 'Ziricote', 'Rosewood',
       'Sapele', 'Cherry', 'Solid Rosewood', 'Ovangkol',
       'Ovangkol, Solid', 'Maple, Solid', 'Okume', 'Basswood', 'Ebony'],
      dtype=object)

In [292]:
df['FRETBOARD'].unique()

array(['Walnut', 'Roseacer', 'Pau Ferro', 'Rosewood', 'Laurel',
       'Ovangkol', 'Ebony', 'Merbau', 'Maple', 'indian Laurel', 'Nandu',
       'Blackwood', 'Ebonol'], dtype=object)

In [293]:
df['FRETS'].unique()

array(['20', '643', '21'], dtype=object)

In [294]:
df['FRETS']=df['FRETS'].astype('int')

In [295]:
for index,value in enumerate(df['FRETS']):
    if value >= 24 :
        df.drop(index, axis=0, inplace=True)

In [296]:
df['COLOUR'].unique()

array(['Black', 'Blue', 'Natural', 'Sunburst', 'Natural Mahogany',
       'Sunburst, High Gloss', 'Natural, High Gloss'], dtype=object)

In [297]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98 entries, 0 to 99
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   MANUFACTURER          98 non-null     object
 1   MODEL                 98 non-null     object
 2   PRICE                 98 non-null     object
 3   PRODUCT_AVAILABILITY  98 non-null     object
 4   GUITAR_RANK           98 non-null     object
 5   YEARS_WARRANTY        98 non-null     object
 6   AVAILABLE SINCE       98 non-null     object
 7   ITEM NUMBER           98 non-null     object
 8   TOP                   98 non-null     object
 9   BACK AND SIDES        98 non-null     object
 10  CUTAWAY               98 non-null     object
 11  FRETBOARD             98 non-null     object
 12  NUT WIDTH IN MM       98 non-null     object
 13  FRETS                 98 non-null     int32 
 14  PICKUPS               98 non-null     object
 15  COLOUR                98 non-null     obje

In [298]:
for index,value in enumerate(df['PRICE']):
        if ',' in value:
             df['PRICE'].replace(value, value.replace(',',''), inplace= True)

In [299]:
df['PRICE'].astype('float')

0     145.0
1      75.0
2     111.0
3     111.0
4      85.0
      ...  
95    655.0
96    239.0
97    329.0
98    319.0
99    599.0
Name: PRICE, Length: 98, dtype: float64

In [300]:
df['GUITAR_RANK']=df['GUITAR_RANK'].astype('int')

In [301]:
df['YEARS_WARRANTY']=df['YEARS_WARRANTY'].astype('int')

In [302]:
df['ITEM NUMBER']=df['ITEM NUMBER'].astype('int')

In [303]:
df['NUT WIDTH IN MM'][0]

'43,00 mm'

In [304]:
df['AVAILABLE SINCE']=pd.to_datetime(df['AVAILABLE SINCE'])